## [Link para documento overleaf](https://www.overleaf.com/project/677fe7fd4bf7cc5092992e57) 

In [0]:
#Bibliotecas
import pandas as pd
from typing import List,Callable

In [0]:
#Funções Intermédiarias da Função Principal
#Pendência -> Trocar as várias querys por a query única feita pelo Cauê e pelo Cícero
def extrair_informacoes(ativo: str) -> List[pd.DataFrame]:
    '''
    Objetivo:
        Extrair informações necessárias para o Cálculo da Régua
    Inputs:
        ativo: Código do ativo que servirá para extrair informações em relação ao book micro e fundo restrito
    Output:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max']
        peso_book_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','book','peso_book']
        credito_aportado: DataFrame com valor de crédito alocado em cada book em cada fundo ['fundo','book','produto',alocado']
        restricoes_book_micro: DataFrame com as restrições book micro [book','fundo',','flag']
        restricoes_fundo_restrito: Dataframe com restrições fundo restrito ['ativo',fundo','flag']
    '''
    #Extraindo inf_fundos
    #Patrimônio Líquido -> Tabela cotas
    pls = spark.sql(
        "SELECT t.Codigo,t.PL,t.Data FROM desafio_kinea.boletagem_cp.cotas t JOIN (SELECT Codigo, MAX(Data) AS data_mais_recente FROM desafio_kinea.boletagem_cp.cotas GROUP BY Codigo) sub ON t.Codigo = sub.Codigo AND t.Data = sub.data_mais_recente;"
    ).toPandas()
    
    #Range mínimo e máximo de crédito -> Tabela range_alocacao
    ranges = spark.sql(
        "SELECT * FROM desafio_kinea.boletagem_cp.range_alocacao"
    ).toPandas()

    #Formando inf_fundos
    pls = pls.rename(columns={'Codigo':'fundo'})
    ranges = ranges.rename(columns={'Fundo':'fundo'})
    inf_fundos = pd.merge(pls,ranges,on='fundo')
    inf_fundos.drop(columns=['Data'],inplace=True)
    
    #Extraindo book_fundos
    peso_book_fundos = spark.sql(
        "SELECT * FROM desafio_kinea.boletagem_cp.pesos_classes"
    ).toPandas().rename(columns={'Fundo':'fundo','classe':'book','peso':'peso_book'})
    peso_book_fundos = peso_book_fundos

    #Extraindo credito_aportado 
    tabela_base = spark.sql("""
        WITH 
        tab_booksoverview AS (

        SELECT
        distinct
        PositionDate
        ,Book          --Book: Classificacao organizacional dos ativos dos fundos
        ,Product       --Product: Produto, é o prório ativo
        ,ProductClass  --ProductClass: Tipo de Produto (Debenture,Fidc, Cri/Cra, etc)
        ,TradingDesk   --Fundo
        ,Position      --Posicao em valor financeiro total do ativo

        FROM
        desafio_kinea.boletagem_cp. booksoverviewposicao_fechamento--Tabela com as posicoes dos fundos


        where (  LOWER(ProductClass) like '%debenture%'  --Apenas Ativos de Crédito Privado
            or LOWER(ProductClass) like '%bonds%' 
            or LOWER(ProductClass) like '%cra%' 
            or LOWER(ProductClass) like '%cri%' 
            or LOWER(ProductClass) like '%funds%' 
            or LOWER(ProductClass) like '%letra%' 
            or LOWER(ProductClass) like '%nota%'
            )

        and (  LOWER(Book) like '%ivan%') --Filtra Books Ivan (gestor do fundo) - Apenas Crédito Privado

        and TradingDesk in ('KCP','RFA','KOP', '846', '134', '678','FRA', 'CPI','PAL','ID2','PID','APO','APP','IRF','KAT','PEM','PDA',"KRF","652","389","348","BVP") --Apenas fundos finais

        )

        , tab_pl as (

        SELECT
        distinct
        Data as PositionDate
        ,Codigo as TradingDesk
        ,PL
        from  desafio_kinea.boletagem_cp.cotas --Tabela com o PL dos fundos
        )

        ,tab_fundos as (
        SELECT
        DISTINCT
        tab_booksoverview.*
        ,tab_pl.PL
        FROM
        tab_booksoverview
        LEFT JOIN
        tab_pl
        ON tab_pl.PositionDate = tab_booksoverview.PositionDate and tab_pl.TradingDesk = tab_booksoverview.TradingDesk
        ) 
        select PositionDate,Book,Product,t.TradingDesk,Position,PL from tab_fundos t JOIN (
        select TradingDesk,MAX(PositionDate) as MaxDate from tab_fundos group by TradingDesk
        ) sub ON t.TradingDesk = sub.TradingDesk AND t.PositionDate = sub.MaxDate
        """
    ).toPandas().groupby(['TradingDesk','Book','Product']).agg({'Position':'sum'}).reset_index().rename(columns={"TradingDesk":'fundo','Book':'book','Product':'produto','Position':'alocado'})

    #Extraindo restricoes_books_micro

    #Extraindo restricao_fundos
    


    #return [inf_fundos,peso_book_fundos,credito_aportado]
    return []

#Fazer essa Função
def redistribuir(alocacao):
    return

#Fazer essa Função
def calculo_posicao(inf_fundos: pd.DataFrame,book_micro: str,) -> pd.DataFrame:
    return

def calculo_regua(inf_fundos: pd.DataFrame, peso_books_fundos:pd.DataFrame, restricoes_books_micro:pd.DataFrame, restrioces_fundo_restrito:pd.DataFrame,credito_total:float) -> pd.DataFrame:
    '''
    Objetivo:
        Cálculo da Régua
    Inputs:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max']
        peso_books_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','book','peso_book']
        credito_total: float com o valor total do crédito 
    Output:
        regua: DataFrame com as informações da régua que contém as alocações ideais ['fundo','book','percentual_alocacao']
    '''

    #Funções Auxiliares
    def percentual_alocacao_range_total(credito_total:float,inf_fundos: pd.DataFrame) -> float:
        inf_fundos['v_min'] = inf_fundos['PL'] * inf_fundos['l_min']
        inf_fundos['v_max'] = inf_fundos['PL'] * inf_fundos['l_max']
        return (credito_total - inf_fundos['v_min'].sum())/ (inf_fundos['v_max'].sum() - inf_fundos['v_min'].sum())
    
    def percentual_credito_relativo_pl(percentual_alocacao_range_total:float,inf_fundo: pd.Series) -> dict:
        return inf_fundo['l_min'] + percentual_alocacao_range_total * (inf_fundo['l_max'] - inf_fundo['l_min'])
    
    def percentual_book_relativo_pl(percentual_credito_relativo_pl:float,peso_book_fundo: float) -> float:
        return percentual_credito_relativo_pl * peso_book_fundo
    
    def valor_book_relativo_pl(percentual_book_relativo_pl:float,pl:float) -> float:
        return pl *percentual_book_relativo_pl

    #Cálculo da Régua
    percentual_alocacao_range_total = percentual_alocacao_range_total(credito_total,inf_fundos)
    
    percentuais_credito_relativo_pl = {}
    for index,row in inf_fundos.iterrows():
        percentuais_credito_relativo_pl[row['fundo']] = percentual_credito_relativo_pl(percentual_alocacao_range_total,row)

    percentuais_books_relativo_pl = pd.DataFrame(columns=['fundo','book','percentual'])
    for fundo,percentual_credito_relativo_pl in percentuais_credito_relativo_pl.items():
        for index,row in peso_books_fundos[peso_books_fundos['fundo'] == fundo].iterrows():
            percentuais_books_relativo_pl.loc[len(percentuais_books_relativo_pl)] = [fundo,row['book'],percentual_book_relativo_pl(percentual_credito_relativo_pl,row['peso_book'])]
    
    valores_books_relativos_pl = pd.DataFrame(columns=['fundo','book','valor'])
    for index,row in percentuais_books_relativo_pl.iterrows():
        valores_books_relativos_pl.loc[len(valores_books_relativos_pl)] = [row['fundo'],row['book'],valor_book_relativo_pl(row['percentual'],inf_fundos[inf_fundos['fundo'] == row['fundo']]['PL'].values[0])]
   
    regua = pd.DataFrame(columns=['fundo','book','percentual_alocacao'])
    books = valores_books_relativos_pl['book'].unique()
    fundos = valores_books_relativos_pl['fundo'].unique()
    somas_books = {}
    for book in books:
        somas_books[book] = valores_books_relativos_pl[valores_books_relativos_pl['book'] == book]['valor'].sum()
    for fundo in fundos:
        for index,row in valores_books_relativos_pl[valores_books_relativos_pl['fundo'] == fundo].iterrows():
            regua.loc[len(regua)] = [fundo,row['book'],row['valor']/somas_books[row['book']]]
    
    #Passar por verificações de book micro e fundo restrito
    #Redistribuir
    return regua

In [0]:
#Função Principal
def solucao(checagens: List[Callable],ordens: pd.DataFrame) -> pd.DataFrame:
    '''
    Função principal que recebe como input:
    1. checagens: Lista de funções geradas pelo ChatGPT
    2. ordens: DataFrame ['Ticker','Amount','Price'] representando uma grupo de ordens a serem boletadas
    e retorna:
    1. ordens_final: DatFrame ['Ticker','Amount','Price','Fundo'] representando quanto irá para cada fundo
    '''
    #1.Etapa - Extrair Informações dos Fundos (Patrimônio Líquido, Range Mínimo e Máximo de Crédito, Pesos de cada Book em cada Fundo)
    extrair_informacoes()
    #2.Etapa -> Calcular a Régua Book Macro
    

    #3 Etapa -> Aplicar Restição de book micro e fundo restrito
   

   #4. etapa -> Passar por funções de verificação

   #4.1 -> Calcular quanto está errado e redistribuir
   #4.1.1 -> Precisamos de um método de otimização? Gradiente ou outrca coisa? Se sim Desenvolver teórioco

   #5 -> Reverificar  e caso passe entregar o resultado final


{'RFA': [6165720738.95119], 'APO': [2279627646.384785], 'ID2': [3488179932.21023], 'APP': [8430325498.580385], 'PID': [3937179801.4903502], 'KAT': [3004467197.6176753], 'KCP': [3424900701.5180855], 'CPI': [54355074.65991], 'BVP': [1191376868.3027952], 'PAL': [18720368.46087], 'IRF': [168954523.486945], '846': [3130395619.174485], '134': [7003331463.740895], '678': [6318358039.8425455], 'KOP': [733760002.8074601], 'FRA': [78590201.440485], 'PDA': [1172652382.054455], 'KRF': [2061515384.744035], '652': [2496049914.0221553], '389': [94841789.981635]}


In [0]:
#Teste da extração de informações
df = extrair_informacoes()[0]
df.head()


,TradingDesk,Book,Product,Position
0,134,Ivan_CRI_DI+_HTM,CRA CORPORATIVO 6.7282% EMISSÃO 85 SÉRIE 3 28D...,3.124919e+06
1,134,Ivan_CRI_DI+_HTM,CRA MONTE CARLO 7.5% EMISSÃO 86 SÉRIE 1 22FEB2029,9.693173e+06
2,134,Ivan_CRI_DI+_HTM,CRA MONTE CARLO 7.5% EMISSÃO 87 SÉRIE 1 22FEB2029,1.830199e+06
3,134,Ivan_CRI_DI+_HTM,CRI 1.25 TRUE SECURITIZADORA SERIE 247 19DEC2029,1.881733e+07
4,134,Ivan_CRI_DI+_HTM,CRI BARI 1.45 22Nov2029,2.325924e+07


In [0]:
#Teste Cálculo Régua - Exemplo feito pelo Ali na Reunião do dia 09/01/2025 (https://drive.google.com/file/d/1O-bVrF1At0xZNQIGmIOQ1Oi7YiSkGD0u/view?usp=sharing)
credito_total = 40
teste_inf_fundos = pd.DataFrame({
        'fundo':['RFA','APO','ID2'],
        'PL':[100,50,70],
        'l_min':[0.1,0.1,0.1],
        'l_max':[0.2,0.2,0.4]
})
teste_books_fundos  = pd.DataFrame({
        'fundo':['RFA','RFA','RFA','APO','APO','APO','ID2','ID2','ID2'],
        'book':['HG','MY','HY','HG','MY','HY','HG','MY','HY'],
        'peso_book':[0.53778,0.1634,0.29882,0.23006,0.433212,0.336729,0.41028,0.20813,0.38159]
})
resposta_nossa = calculo_regua(teste_inf_fundos,teste_books_fundos,credito_total)
resposta_ali = pd.DataFrame({
        'fundo':['RFA','RFA','RFA','APO','APO','APO','ID2','ID2','ID2'],
        'book':['HG','MY','HY','HG','MY','HY','HG','MY','HY'],
        'dist_regua':[0.4753,0.2623,0.3275,0.1017,0.3478,0.1845,0.4230,0.3899,0.4879]
        
})
print(resposta_nossa['percentual_alocacao'])
print(resposta_ali['dist_regua'])

0    0.475293
1    0.262353
2    0.327520
3    0.101664
4    0.347780
5    0.184535
6    0.423043
7    0.389866
8    0.487946
Name: percentual_alocacao, dtype: float64
0    0.4753
1    0.2623
2    0.3275
3    0.1017
4    0.3478
5    0.1845
6    0.4230
7    0.3899
8    0.4879
Name: dist_regua, dtype: float64
